# IMPORTS

In [1]:
import pandas as pd

# import json
# from collections import defaultdict

import my_pickle as mp
import my_ect_split as ms
import my_dataframes as md

from importlib import reload

In [ ]:
reload(md)

# READ IN

In [2]:
# read in message data
message_df = md.get_messages()
convo_df = md.get_conversations(message_df)
last_df = md.get_lastmessage()

0
if this number is not zero, you have a problem: 0


# ECT SPLIT

In [3]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
pd.to_datetime(cutoff*1000000)

Timestamp('2017-10-02 01:35:58.644000')

In [4]:
# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
pd.to_datetime(safe_convo_df.timestamp.max()*1000000)

Timestamp('2017-10-02 01:35:58.644000')

# COMBINE USER AND MESSAGE FILES

NB: Have to do after ECT split!

In [5]:
responsiveness_df = md.get_responsiveness(convo_df)

In [6]:
# find average attractiveness
average_attractiveness = responsiveness_df[responsiveness_df.attractiveness >0].mean()['attractiveness']
average_selectivity = responsiveness_df[responsiveness_df.selectivity >0].mean()['selectivity']

In [8]:
# Combine with user table
user_df = md.get_user_data()
user_df = user_df.join(responsiveness_df)
user_df = user_df.fillna({'messages_sent' : 0,'responses_received': 0,'attractiveness':average_attractiveness,
                          'messages_received': 0,'responses_sent': 0, 'selectivity':average_selectivity})
user_df.head(2)
# issue where 0 and 0 can be 0 or average b/c of way joined. ignore for now.

,created,updated,about,has_about,len_about,available,birthday,age,collegeId,emailVerified,...,roomTypeId,smokingOk,term,work,messages_sent,responses_received,attractiveness,messages_received,responses_sent,selectivity
uid,,,,,,,,,,,,,,,,,,,,,
0a9yOPKFSH,2017-06-19 03:31:08,2018-01-04 21:24:41,,False,0,nan,1992-01-01,26.0,NaN,1,...,2.0,0.0,12.0,SEG,7.0,2.0,0.285714,9.0,2.0,0.777778
0owxBbcyZ6,2017-10-12 00:32:29,2018-01-04 03:05:50,,False,0,nan,1974-01-01,44.0,NaN,1,...,NaN,1.0,NaN,WA State,0.0,0.0,0.000000,7.0,0.0,0.000000


In [9]:
print(len(user_df))
print(user_df.onboarded.sum())

32436
32436.0


# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [10]:
mp.pickle_it(user_df,'user_df')
# mp.pickle_it(message_df,'message_df')  NOT ECT SAFE!
mp.pickle_it(safe_convo_df,'convo_df')  # make sure pickling safe stuff
# pickle_it(lastmessage_df,'lastmessage_df') NOT ECT SAFE!